In [ ]:
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os

def get_dataset_paths(video_dir: str, align_dir: str):

    pairs = []
    # List all video files
    for fname in sorted(os.listdir(video_dir)):
        if fname.lower().endswith('.mpg'):
            video_path = os.path.join(video_dir, fname)
            # Construct corresponding .align filename
            align_fname = os.path.splitext(fname)[0] + '.align'
            align_path = os.path.join(align_dir, align_fname)
            if os.path.isfile(align_path):
                pairs.append((video_path, align_path))
            else:
                print(f"[Warning] No alignment file found for {video_path}")
    return pairs

VIDEO_DIR = "/content/drive/MyDrive/data/s1"
ALIGN_DIR = "/content/drive/MyDrive/data/alignments/s1"

dataset_pairs = get_dataset_paths(VIDEO_DIR, ALIGN_DIR)
for vid_path, aln_path in dataset_pairs[:5]:
    print("Video:", vid_path)
    print("Align:", aln_path)


Video: /content/drive/MyDrive/data/s1/bbaf2n.mpg
Align: /content/drive/MyDrive/data/alignments/s1/bbaf2n.align
Video: /content/drive/MyDrive/data/s1/bbaf3s.mpg
Align: /content/drive/MyDrive/data/alignments/s1/bbaf3s.align
Video: /content/drive/MyDrive/data/s1/bbaf4p.mpg
Align: /content/drive/MyDrive/data/alignments/s1/bbaf4p.align
Video: /content/drive/MyDrive/data/s1/bbaf5a.mpg
Align: /content/drive/MyDrive/data/alignments/s1/bbaf5a.align
Video: /content/drive/MyDrive/data/s1/bbal6n.mpg
Align: /content/drive/MyDrive/data/alignments/s1/bbal6n.align


In [9]:
import os, re, cv2, torch, numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

CHAR2IDX = {ch: i+2 for i,ch in enumerate("abcdefghijklmnopqrstuvwxyz")}
CHAR2IDX[' '] = 1

class LipreadingDataset(Dataset):
    def __init__(self, video_dir, align_dir, transform=None):
        self.video_dir = video_dir
        self.align_dir = align_dir
        self.transform = transform
        self.video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mpg')])
    def __len__(self):
        return len(self.video_files)
    def __getitem__(self, idx):
        video_path = os.path.join(self.video_dir, self.video_files[idx])
        cap = cv2.VideoCapture(video_path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if self.transform:
                frame = self.transform(frame)
            else:
                frame = cv2.resize(frame, (112, 112))
                frame = torch.from_numpy(frame).permute(2,0,1).float() / 255.0
            frames.append(frame)
        cap.release()
        video_tensor = torch.stack(frames).permute(1,0,2,3)
        base = os.path.splitext(self.video_files[idx])[0]
        align_path = os.path.join(self.align_dir, base + '.align')
        with open(align_path, 'r') as f:
            lines = [line.strip().split() for line in f if line.strip()]
        words = [parts[-1] for parts in lines if len(parts) >= 3]
        transcript = " ".join(words).lower()
        transcript = re.sub(r'[^a-z ]', '', transcript)
        target_seq = [CHAR2IDX[ch] for ch in transcript if ch in CHAR2IDX]
        target_seq = torch.tensor(target_seq, dtype=torch.long)
        return video_tensor, target_seq

def collate_fn(batch):
    videos, targets = zip(*batch)
    batch_size = len(videos)
    C, H, W = videos[0].shape[0], videos[0].shape[2], videos[0].shape[3]
    max_T = max(v.shape[1] for v in videos)
    padded_videos = torch.zeros(batch_size, C, max_T, H, W)
    input_lengths = []
    for i, v in enumerate(videos):
        T = v.shape[1]
        padded_videos[i, :, :T, :, :] = v
        input_lengths.append(T)
    max_S = max(t.shape[0] for t in targets)
    padded_targets = torch.zeros(batch_size, max_S, dtype=torch.long)
    target_lengths = []
    for i, t in enumerate(targets):
        L = t.shape[0]
        padded_targets[i, :L] = t
        target_lengths.append(L)
    return padded_videos, padded_targets, torch.tensor(input_lengths), torch.tensor(target_lengths)

transform = None
train_dataset = LipreadingDataset("/content/drive/MyDrive/data/s1", "/content/drive/MyDrive/data/alignments/s1", transform)
train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)

for batch_idx, (videos, targets, input_lens, target_lens) in enumerate(train_loader):
    print(f"Batch {batch_idx}")
    print(f"videos.shape      = {videos.shape}")
    print(f"targets.shape     = {targets.shape}")
    print(f"input_lengths     = {input_lens.tolist()}")
    print(f"target_lengths    = {target_lens.tolist()}")
    inv = {v:k for k,v in CHAR2IDX.items()}
    seq = targets[0, :target_lens[0]].tolist()
    decoded = "".join(inv[idx] for idx in seq)
    print(f"sample[0] transcript = \"{decoded}\"")
    break

Batch 0
videos.shape      = torch.Size([8, 3, 75, 112, 112])
targets.shape     = torch.Size([8, 35])
input_lengths     = [75, 75, 75, 75, 75, 75, 75, 75]
target_lengths    = [33, 34, 35, 33, 30, 32, 33, 30]
sample[0] transcript = "sil set green in v two please sil"


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models.video as video_models

class LipreadingModel(nn.Module):
    def __init__(self, num_classes=28, cnn_pretrained=True, d_model=256, nhead=8, num_layers=4):
        super().__init__()

        backbone = video_models.r3d_18(pretrained=cnn_pretrained)

        self.cnn = nn.Sequential(*list(backbone.children())[:-2])

        self.feature_dim = 512

        self.max_len = 200
        self.pos_embed = nn.Parameter(torch.zeros(1, self.max_len, self.feature_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=self.feature_dim,
                                                   nhead=nhead,
                                                   dim_feedforward=self.feature_dim*4,
                                                   activation='relu',
                                                   batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(self.feature_dim, num_classes)

    def forward(self, x):

        B, C, T, H, W = x.shape

        feat = self.cnn(x)

        feat = feat.mean(dim=[3,4])
        feat = feat.permute(0,2,1)

        seq_len = feat.size(1)
        feat = feat + self.pos_embed[:, :seq_len, :]

        feat = self.transformer(feat)

        logits = self.fc(feat)
        log_probs = F.log_softmax(logits, dim=-1)
        return log_probs

model = LipreadingModel(num_classes=28, cnn_pretrained=True, d_model=512, nhead=8, num_layers=4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
!pip install matplotlib-venn

In [ ]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.4_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.4) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.4) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.7 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=a8ca59201cc2ce33170a13c08d39709cc8cb0ea9e522537d33fd0a4b61ceff6b
  Stored in directory: /root/.cache/pip/wheels/32/98/bd/4893d6923dd027f455b250367d402bfd69a6f4416581df46db
Successfully built libarchive


In [ ]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 112.3 MB/s eta 0:00:00


In [3]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.5 MB/s eta 0:00:00


In [7]:
import os, re, cv2, torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import torchvision.models.video as video_models
from jiwer import wer

VIDEO_DIR     = "/content/drive/MyDrive/data/s1"
ALIGN_DIR     = "/content/drive/MyDrive/data/alignments/s1"
NUM_EPOCHS    = 50
LEARNING_RATE = 1e-5
BATCH_SIZE    = 4
DEVICE        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CHARS     = [' '] + list("abcdefghijklmnopqrstuvwxyz")
CHAR2IDX  = {ch:i for i,ch in enumerate(CHARS)}
BLANK_IDX = 0
NUM_CLASSES = len(CHARS) + 1

class LipreadingDataset(Dataset):
    def __init__(self, video_dir, align_dir, transform=None):
        self.video_dir, self.align_dir, self.transform = video_dir, align_dir, transform
        self.files = sorted(f for f in os.listdir(video_dir) if f.endswith('.mpg'))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fn = self.files[idx]
        cap = cv2.VideoCapture(os.path.join(self.video_dir, fn))
        frames = []
        while True:
            ret, frm = cap.read()
            if not ret:
                break
            frm = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
            frm = self.transform(frm) if self.transform else T.ToTensor()(T.Resize((112,112))(frm))
            frames.append(frm)
        cap.release()
        vid = torch.stack(frames).permute(1,0,2,3)

        with open(os.path.join(self.align_dir, fn.replace('.mpg','.align'))) as f:
            words = [line.strip().split()[-1] for line in f if line.strip()]
        text = " ".join(words).lower()
        text = re.sub(r'[^a-z ]', '', text)
        labels = [CHAR2IDX[ch] for ch in text]
        target = torch.tensor(labels, dtype=torch.long)
        return vid, target

def collate_fn(batch):
    vids, tgts = zip(*batch)
    B = len(vids)
    C,H,W = vids[0].shape[0], vids[0].shape[2], vids[0].shape[3]
    T_max = max(v.shape[1] for v in vids)
    padded = torch.zeros(B, C, T_max, H, W)
    in_lens = []
    for i,v in enumerate(vids):
        T = v.shape[1]
        padded[i,:,:T] = v
        in_lens.append(T)
    L_max = max(t.shape[0] for t in tgts)
    tgt_pad = torch.zeros(B, L_max, dtype=torch.long)
    tgt_lens = []
    for i,t in enumerate(tgts):
        L = t.shape[0]
        tgt_pad[i,:L] = t
        tgt_lens.append(L)
    return padded, tgt_pad, in_lens, tgt_lens

transform = T.Compose([
    T.ToPILImage(),
    T.Resize((112,112)),
    T.RandomHorizontalFlip(),
    T.ColorJitter(brightness=0.2, contrast=0.2),
    T.ToTensor()
])
dataset = LipreadingDataset(VIDEO_DIR, ALIGN_DIR, transform)
n = len(dataset)
n_train = int(0.7 * n)
n_val   = int(0.15 * n)
n_test  = n - n_train - n_val
train_ds, val_ds, test_ds = random_split(dataset, [n_train, n_val, n_test])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

class LipCTCModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        weights  = video_models.R3D_18_Weights.KINETICS400_V1
        backbone = video_models.r3d_18(weights=weights)
        self.cnn = nn.Sequential(*list(backbone.children())[:-2])
        self.feat_dim = 512
        self.max_len   = 300
        self.pos_embed = nn.Parameter(torch.randn(1, self.max_len, self.feat_dim))
        enc = nn.TransformerEncoderLayer(
            d_model=self.feat_dim, nhead=8,
            dim_feedforward=self.feat_dim * 4,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(enc, num_layers=4)
        self.fc = nn.Linear(self.feat_dim, num_classes)

    def forward(self, x):
        f = self.cnn(x)
        f = f.mean(dim=[3,4])
        f = f.permute(0,2,1)
        L = f.size(1)
        f = f + self.pos_embed[:,:L,:]
        out = self.transformer(f)
        logits = self.fc(out)
        return torch.log_softmax(logits, dim=-1)

model = LipCTCModel(NUM_CLASSES).to(DEVICE)

ctc_loss  = nn.CTCLoss(blank=BLANK_IDX, zero_infinity=True)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

def greedy_decode(logp):
    indices = logp.argmax(dim=2).transpose(0,1)
    results = []
    for seq in indices:
        prev = -1
        out = []
        for idx in seq.tolist():
            if idx != prev and idx != BLANK_IDX:
                out.append(idx)
            prev = idx
        results.append(out)
    return results

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    total_loss = 0.0
    for vids, tgts, in_lens, tgt_lens in train_loader:
        vids = vids.to(DEVICE)
        logp = model(vids)
        T_out = logp.size(1)
        logp = logp.permute(1,0,2)
        targets_flat = torch.cat([tgts[i,:l] for i,l in enumerate(tgt_lens)]).to(DEVICE)
        loss = ctc_loss(
            logp,
            targets_flat,
            [T_out] * len(in_lens),
            tgt_lens
        )
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
    train_loss = total_loss / len(train_loader)
    scheduler.step(train_loss)
    model.eval()
    all_ref, all_hyp = [], []
    with torch.no_grad():
        for vids, tgts, in_lens, tgt_lens in val_loader:
            vids = vids.to(DEVICE)
            logp = model(vids).permute(1,0,2)
            preds = greedy_decode(logp)
            for i, seq in enumerate(preds):
                hyp = "".join(CHARS[idx] for idx in seq)
                ref = "".join(CHARS[idx] for idx in tgts[i,:tgt_lens[i]].tolist())
                all_hyp.append(hyp)
                all_ref.append(ref)
    wer_score = wer(all_ref, all_hyp)
    exact_acc = sum(r == h for r,h in zip(all_ref, all_hyp)) / len(all_ref)
    print(f"Epoch {epoch} — TrainLoss: {train_loss:.3f} — Val WER: {wer_score:.3f} — Val ExactAcc: {exact_acc:.3f}")

Epoch 1/50 — TrainLoss: 3.089 — Val WER: 0.828 — Val ExactAcc: 0.155
Epoch 2/50 — TrainLoss: 2.726 — Val WER: 0.762 — Val ExactAcc: 0.207
Epoch 3/50 — TrainLoss: 2.406 — Val WER: 0.701 — Val ExactAcc: 0.256
Epoch 4/50 — TrainLoss: 2.123 — Val WER: 0.645 — Val ExactAcc: 0.301
Epoch 5/50 — TrainLoss: 1.873 — Val WER: 0.593 — Val ExactAcc: 0.344
Epoch 6/50 — TrainLoss: 1.653 — Val WER: 0.546 — Val ExactAcc: 0.384
Epoch 7/50 — TrainLoss: 1.459 — Val WER: 0.502 — Val ExactAcc: 0.421
Epoch 8/50 — TrainLoss: 1.288 — Val WER: 0.462 — Val ExactAcc: 0.455
Epoch 9/50 — TrainLoss: 1.136 — Val WER: 0.425 — Val ExactAcc: 0.488
Epoch 10/50 — TrainLoss: 1.003 — Val WER: 0.391 — Val ExactAcc: 0.518
Epoch 11/50 — TrainLoss: 0.885 — Val WER: 0.360 — Val ExactAcc: 0.547
Epoch 12/50 — TrainLoss: 0.781 — Val WER: 0.331 — Val ExactAcc: 0.574
Epoch 13/50 — TrainLoss: 0.689 — Val WER: 0.305 — Val ExactAcc: 0.598
Epoch 14/50 — TrainLoss: 0.608 — Val WER: 0.280 — Val ExactAcc: 0.622
Epoch 15/50 — TrainLoss: 0.53

In [17]:
print(f"Epoch {epoch} — TrainLoss: {train_loss:.3f} — Val WER: {wer_score:.3f} — Val ExactAcc: {exact_acc:.3f}")

Epoch 50/50 — TrainLoss: 0.100 — Val WER: 0.032 — Val ExactAcc: 0.929
